In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import pickle
import requests
import os
import pathlib

In [23]:
driver = webdriver.Chrome('../../chromedriver')

In [22]:
whitelist = ['minimal', 'landscape', 'abstract']
blacklist = ['phone']
WEBSITE_PAUSE_TIME = 2
IMAGE_PAUSE_TIME = 0.5
SCROLL_PAUSE_TIME = 1

In [15]:
image_urls=[]
viewed_pages = set()
image_categories={}

queue = [
    "https://wallpaperaccess.com/minimal-landscape",
    "https://wallpaperaccess.com/sunset-minimal",
    "https://wallpaperaccess.com/minimal-city",
    "https://wallpaperaccess.com/minimal-phone",
    "https://wallpaperaccess.com/modern-minimal"
]

In [6]:
# DOESN'T WORK

try:
    with open('/data1/wallpaperAccessPhotos.pkl', 'rb') as f:
        image_urls = pickle.load(f)
        print(len(image_urls))
        
    with open('/data1/wallpaperAccessPhotoCategories.pkl', 'rb') as f:
        image_categories = pickle.load(f)
        print(len(image_categories))
        
        viewed_pages = set(image_categories.keys())
        print(len(viewed_pages))
        
#     print('Avg imgs per cat', len(image_urls)/len(viewed_pages))

except Exception:
    print('Does not exist')
    
list(image_categories.keys())[2]

30572
25848
25848


'https://wallpaperaccess.com/full/99790.jpg'

In [19]:
with open('/data1/wallpaperAccessPhotos.pkl', 'rb') as f:
    image_urls = pickle.load(f)
    print(len(image_urls))

root_url = 'https://wallpaperaccess.com/'
data_path = pathlib.Path('/data1', 'Raw', 'WallpaperAccess')
data_directories = [f for f in data_path.iterdir() if f.is_dir()]
print(len(data_directories))

viewed_pages = set(['{}{}'.format(root_url, str(d).split('/')[-1]) for d in data_directories][0])

queue = [
    'https://wallpaperaccess.com/august-2020-calendar',
    'https://wallpaperaccess.com/acrylic',
    'https://wallpaperaccess.com/acrylic-paint'
]

30572
618


In [20]:
def scroll_to_bottom(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
# page_limit = 200*10
i=0
while True:
#     if i > page_limit:
#         break
    
    if len(queue) <= 0:
        break
        
    url = queue.pop(0)
    
    if url in viewed_pages:
        continue
    
    driver.get(url)
    
    time.sleep(WEBSITE_PAUSE_TIME)
    scroll_to_bottom(driver)
    time.sleep(WEBSITE_PAUSE_TIME)
    
    imgs = driver.find_elements(By.XPATH, '//img[@class="thumb ads_popup"]')
    
    image_category = url.split('/')[-1]
    
    if(len(imgs) == 0):
        time.sleep(WEBSITE_PAUSE_TIME*5)
        imgs = driver.find_elements(By.XPATH, '//img[@class="thumb ads_popup"]')
    
    for img in imgs:
        img_url = img.get_attribute('src')
        image_urls.append(img_url)
        
        if img_url not in image_categories:
            image_categories[img_url] = []
        image_categories[img_url].append(url)
        
        img_id = img_url.split('/')[-1]
        path = pathlib.Path('/data1', 'Raw', 'WallpaperAccess', image_category, img_id)
        path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(path,'wb') as f:
            f.write(requests.get(img_url).content)
            f.close()
        
        time.sleep(IMAGE_PAUSE_TIME)
        
        
    print(url, len(imgs))
    print(len(image_urls))
        
    related_wallpapers = driver.find_elements(By.XPATH, '//div[@class="column collection_thumb"]/a')
    
    for link in related_wallpapers:
        wp_link = link.get_attribute('href')
        eval_val = 0
        
        for whitelist_word in whitelist:
            if whitelist_word in wp_link:
                eval_val += 1
        for blacklist_word in blacklist:
            if blacklist_word in wp_link:
                eval_val -= 1
        if eval_val > 0:
            queue.insert(0, wp_link)
        else:
            queue.append(wp_link)
        
    viewed_pages.add(url)
    
    i+=1
    if (i%50) == 0:
        with open('/data1/wallpaperAccessPhotos.pkl', 'wb') as f:
            pickle.dump(image_urls, f)

        with open('/data1/wallpaperAccessPhotoCategories.pkl', 'wb') as f:
            pickle.dump(image_categories, f)
            
    time.sleep(WEBSITE_PAUSE_TIME)

https://wallpaperaccess.com/vintage-abstract 49
62485
https://wallpaperaccess.com/african-abstract 12
62497
https://wallpaperaccess.com/greek-abstract 5
62502
https://wallpaperaccess.com/abstract-lion-hd 42
62544
https://wallpaperaccess.com/abstract-lion-art-desktop 42
62586
https://wallpaperaccess.com/african-abstract-art 40
62626
https://wallpaperaccess.com/abstract-watercolor 47
62673
https://wallpaperaccess.com/modern-religious-art-abstract 49
62722
https://wallpaperaccess.com/abstract-art-art 52
62774
https://wallpaperaccess.com/3d-modern-abstract-art 31
62805
https://wallpaperaccess.com/pink-abstract-art 39
62844
https://wallpaperaccess.com/5d-abstract 24
62868
https://wallpaperaccess.com/pink-and-grey-abstract 10
62878
https://wallpaperaccess.com/red-and-white-abstract 49
62927
https://wallpaperaccess.com/abstract-pop-art 35
62962
https://wallpaperaccess.com/justin-maller-4k-abstract 63
63025
https://wallpaperaccess.com/justin-maller-ghosts-4k-abstract 80
63105
https://wallpaper

https://wallpaperaccess.com/fine-art-desktop 62
70525
https://wallpaperaccess.com/famous-art-desktop 43
70568
https://wallpaperaccess.com/art-deco-desktop 43
70611
https://wallpaperaccess.com/vintage-desktop 51
70662
https://wallpaperaccess.com/funny-desktop 37
70699
https://wallpaperaccess.com/cool-orange 51
70750
https://wallpaperaccess.com/animated-desktop 47
70797
https://wallpaperaccess.com/hero-desktop 46
70843
https://wallpaperaccess.com/cool-modern-desktop 60
70903
https://wallpaperaccess.com/cool-architecture-desktop 52
70955
https://wallpaperaccess.com/cool-water-desktop 57
71012
https://wallpaperaccess.com/black-hd-desktop 51
71063
https://wallpaperaccess.com/autumn-desktop 49
71112
https://wallpaperaccess.com/cool-black 70
71182
https://wallpaperaccess.com/creepy 56
71238
https://wallpaperaccess.com/most-scary 36
71274
https://wallpaperaccess.com/blue-and-gold-desktop 53
71327
https://wallpaperaccess.com/retro-desktop 51
71378
https://wallpaperaccess.com/spring-desktop 69
7

https://wallpaperaccess.com/mountain-scenery 52
78015
https://wallpaperaccess.com/nordic-winter 44
78059
https://wallpaperaccess.com/norway-forest 51
78110
https://wallpaperaccess.com/norway-desktop 71
78181
https://wallpaperaccess.com/norway-sunset 49
78230
https://wallpaperaccess.com/beautiful-mountains 50
78280
https://wallpaperaccess.com/norway-scenery 50
78330
https://wallpaperaccess.com/lake-fall-nature 53
78383
https://wallpaperaccess.com/church 52
78435
https://wallpaperaccess.com/asia-hd 52
78487
https://wallpaperaccess.com/japanese 63
78550
https://wallpaperaccess.com/seven-continents 5
78555
https://wallpaperaccess.com/oriental 51
78606
https://wallpaperaccess.com/tokyo 51
78657
https://wallpaperaccess.com/chinese 47
78704
https://wallpaperaccess.com/japan 60
78764
https://wallpaperaccess.com/pacific-northwest-coast 50
78814
https://wallpaperaccess.com/pacific-northwest-art 63
78877
https://wallpaperaccess.com/pacific-northwest-camping 46
78923
https://wallpaperaccess.com/pa

https://wallpaperaccess.com/magical-forest 59
85851
https://wallpaperaccess.com/8k-nature 21
85872
https://wallpaperaccess.com/8k-water 13
85885
https://wallpaperaccess.com/8k-ultra-hd-gaming 9
85894
https://wallpaperaccess.com/inspirational-work-quotes 56
85950
https://wallpaperaccess.com/inspiring 50
86000
https://wallpaperaccess.com/inspirational-quotes-motivational 50
86050
https://wallpaperaccess.com/inspirational-quotes-iphone 47
86097
https://wallpaperaccess.com/inspirational-nature 28
86125
https://wallpaperaccess.com/awesome-anime 69
86194
https://wallpaperaccess.com/asian-art 76
86270
https://wallpaperaccess.com/china 74
86344
https://wallpaperaccess.com/ancient-chinese 49
86393
https://wallpaperaccess.com/ancient-china 52
86445
https://wallpaperaccess.com/china-art 37
86482
https://wallpaperaccess.com/traditional-oriental 39
86521
https://wallpaperaccess.com/most-beautiful-desktop 34
86555
https://wallpaperaccess.com/high-resolution-christmas 44
86599
https://wallpaperaccess

https://wallpaperaccess.com/black-dragon-ball-z 41
93303
https://wallpaperaccess.com/goku-ssj-god 52
93355
https://wallpaperaccess.com/ui-goku 53
93408
https://wallpaperaccess.com/hd-superhero 59
93467
https://wallpaperaccess.com/marvel-shield 46
93513
https://wallpaperaccess.com/marvel-simple 48
93561
https://wallpaperaccess.com/marvel-black 54
93615
https://wallpaperaccess.com/white-minimalist-iphone 49
93664
https://wallpaperaccess.com/minimalist-firefox 10
93674
https://wallpaperaccess.com/minimalist-iphone-6 50
93724
https://wallpaperaccess.com/contemporary 45
93769
https://wallpaperaccess.com/minimalist-black-phone 34
93803
https://wallpaperaccess.com/beautiful-widescreen-desktop 62
93865
https://wallpaperaccess.com/humor-and-funny 48
93913
https://wallpaperaccess.com/clean 52
93965
https://wallpaperaccess.com/ambient 49
94014
https://wallpaperaccess.com/blur 51
94065
https://wallpaperaccess.com/8-bit 40
94105
https://wallpaperaccess.com/vaporwave 30
94135
https://wallpaperaccess

https://wallpaperaccess.com/quote 53
100436
https://wallpaperaccess.com/minmalist 51
100487
https://wallpaperaccess.com/ninja 44
100531
https://wallpaperaccess.com/4k-phone 53
100584
https://wallpaperaccess.com/the-end-4k-ultra 57
100641
https://wallpaperaccess.com/ultra-4k-abstract 49
100690
https://wallpaperaccess.com/4k-ultra-hd-planets 69
100759
https://wallpaperaccess.com/car-iphone 46
100805
https://wallpaperaccess.com/race-car 38
100843
https://wallpaperaccess.com/art-2560x1080 37
100880
https://wallpaperaccess.com/2560x1080-red 31
100911
https://wallpaperaccess.com/cool-2560x1080 43
100954
https://wallpaperaccess.com/2560x1080-ultra-wide 28
100982
https://wallpaperaccess.com/2560x1080-black 34
101016
https://wallpaperaccess.com/ultrawide 52
101068
https://wallpaperaccess.com/3840x2160 65
101133
https://wallpaperaccess.com/modern-geometric 17
101150
https://wallpaperaccess.com/modern 67
101217
https://wallpaperaccess.com/purple-geometric 44
101261
https://wallpaperaccess.com/sim

https://wallpaperaccess.com/summer-nature-desktop 47
107712
https://wallpaperaccess.com/dark-green 58
107770
https://wallpaperaccess.com/dark-space 50
107820
https://wallpaperaccess.com/gold-pattern 38
107858
https://wallpaperaccess.com/gold-glitter 48
107906
https://wallpaperaccess.com/black-and-teal 51
107957
https://wallpaperaccess.com/dark-teal 53
108010
https://wallpaperaccess.com/flower 47
108057
https://wallpaperaccess.com/red-nvidia 29
108086
https://wallpaperaccess.com/red-corsair 21
108107
https://wallpaperaccess.com/blue-nature 48
108155
https://wallpaperaccess.com/blue-water 52
108207
https://wallpaperaccess.com/cool-blue-abstract-iphone 36
108243
https://wallpaperaccess.com/black-and-white-desktop 69
108312
https://wallpaperaccess.com/hd-desktop 52
108364
https://wallpaperaccess.com/awesome-rainbow 54
108418
https://wallpaperaccess.com/black-abstract-phone 21
108439
https://wallpaperaccess.com/cool-black-abstract-iphone 51
108490
https://wallpaperaccess.com/colorful-deskto

https://wallpaperaccess.com/avengers-desktop 30
115024
https://wallpaperaccess.com/justice-league-new-52 63
115087
https://wallpaperaccess.com/wonder-woman-justice-league 52
115139
https://wallpaperaccess.com/justice-league-logo 46
115185
https://wallpaperaccess.com/justice-league-computer 56
115241
https://wallpaperaccess.com/captain-america-desktop 43
115284
https://wallpaperaccess.com/ww2-captain-america 54
115338
https://wallpaperaccess.com/captain-america-cartoon 49
115387
https://wallpaperaccess.com/captain-america-shield 35
115422
https://wallpaperaccess.com/star-wars-black 52
115474
https://wallpaperaccess.com/millennium-falcon 59
115533
https://wallpaperaccess.com/hoth 44
115577
https://wallpaperaccess.com/luke-skywalker 47
115624
https://wallpaperaccess.com/swtor 49
115673
https://wallpaperaccess.com/star-wars 103
115776
https://wallpaperaccess.com/blue-marvel 59
115835
https://wallpaperaccess.com/black-widow-4k 41
115876
https://wallpaperaccess.com/black-widow-logo 30
115906

In [31]:
with open('/data1/wallpaperAccessPhotos.pkl', 'wb') as f:
    pickle.dump(image_urls, f)

with open('/data1/wallpaperAccessPhotoCategories.pkl', 'wb') as f:
    pickle.dump(image_categories, f)

10000